In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import *
import pickle
import webbrowser

In [3]:
# Importing the dataset

books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

books.dropna(inplace=True)

C:\Users\siddh\AppData\Local\Temp\ipykernel_15368\1660239959.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'Users.csv'


Models
Popularity Based Approach
The

weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
where:

R = average for the movie (mean)
v = number of votes for the movie
m = minimum votes required to be listed in the Top 250 
C = the mean vote across the whole report ```

In [5]:
# removing all the books whose ISBN is not 10 digits long

books = books[books["ISBN"].apply(lambda x: len(x) == 10)]

unique_ISBN = set(books["ISBN"].unique())

# removing all the ISBNs that are not in the books dataset

ratings = ratings[ratings["ISBN"].isin(unique_ISBN)]

# removing all the users who rated 0 to books

ratings = ratings[ratings["bookRating"] != 0]

In [6]:

# makeing a dataframe which stores the avg rating of a book and also the number of ratings it has got

avg_rating = pd.DataFrame()
avg_rating["ISBN"] = ratings.groupby("ISBN")["bookRating"].mean().index
avg_rating["avg_rating"] = ratings.groupby("ISBN")["bookRating"].mean().values
avg_rating["num_ratings"] = ratings.groupby("ISBN")["bookRating"].count().values

avg_rating.reset_index(inplace=True)


avg_rating.sort_values("num_ratings", ascending=False)
# sum(avg_rating["num_ratings"])

,index,ISBN,avg_rating,num_ratings
21945,21945,0316666343,8.185290,707
117744,117744,0971880107,4.390706,581
38819,38819,0385504209,8.435318,487
18058,18058,0312195516,8.182768,383
4131,4131,0060928336,7.887500,320
...,...,...,...,...
63117,63117,0553100165,8.000000,1
63115,63115,0553100130,8.000000,1
63109,63109,0553099817,8.000000,1
63108,63108,0553099809,7.000000,1


In [7]:

# weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
# where:

# R = average for the movie (mean)
# v = number of votes for the movie
# m = minimum votes required to be listed in the Top 250 
# C = the mean vote across the whole report

m = avg_rating["num_ratings"].quantile(0.99) # top 150 books
C = ratings["bookRating"].mean()

def weighted_rating(x, m = m, C=C):
    v = x["num_ratings"]
    R = x["avg_rating"]
    return (v/(v+m) * R) + (m/(m+v) * C)


avg_rating["weighted_rating"] = avg_rating.apply(weighted_rating, axis=1)

In [8]:
avg_rating.sort_values("weighted_rating", ascending=False,inplace=True)
avg_rating.drop(["index"], axis=1, inplace=True)

In [10]:
avg_rating.head(10)

,ISBN,avg_rating,num_ratings,weighted_rating
46134,0439139597,9.262774,137,9.018886
24538,0345339738,9.402597,77,8.980600
46424,043935806X,9.033981,206,8.887133
46124,0439136369,9.082707,133,8.860131
68897,059035342X,8.939297,313,8.845818
46123,0439136350,9.035461,141,8.830549
50044,0446310786,8.943925,214,8.811095
24537,0345339711,9.120482,83,8.785426
68896,0590353403,8.983193,119,8.755529
46489,0439425220,9.869565,23,8.724268


In [11]:

avg_rating.to_csv("avg_rating.csv", index=False)


book covers of top rated books according to the model

In [13]:
l = avg_rating.sort_values("weighted_rating", ascending=False).head(10)["ISBN"].values

In [14]:
#show image of the book cover

from IPython.display import Image

for i in l:
    display(Image(url=books[books["ISBN"] == i]["imageURLM"].values[0]))
    print(books[books["ISBN"] == i]["bookTitle"].values[0])

Harry Potter and the Goblet of Fire (Book 4)


The Return of the King (The Lord of the Rings, Part 3)


Harry Potter and the Order of the Phoenix (Book 5)


Harry Potter and the Prisoner of Azkaban (Book 3)


Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))


Harry Potter and the Prisoner of Azkaban (Book 3)


To Kill a Mockingbird


The Two Towers (The Lord of the Rings, Part 2)


Harry Potter and the Sorcerer's Stone (Book 1)


Harry Potter and the Chamber of Secrets Postcard Book


Collaborative Filtering
A recommendation technique that leverages the collective behavior and preferences of users to make personalized recommendations.

Steps involved in collaborative filtering:

Data representation: Create a table with users as rows and items (ISBNs) as columns to capture user-item interactions or ratings.

Standardization: Normalize the ratings within each user to remove biases and bring them to a common scale.

Similarity calculation: Compute item-item similarity using metrics like cosine similarity based on user ratings or interactions.

Recommendation generation: Find the most similar items to a given item and recommend them to users who have interacted with the original item.

Collaborative filtering taps into the idea that users with similar tastes for certain items are likely to have similar tastes for other items, providing personalized recommendations based on user behavior and preferences.

In [17]:
# list of top 600 books according to no. of people who rated it

top_600 = avg_rating.sort_values("num_ratings", ascending=False).head(1000)["ISBN"].values

In [18]:
top_600 = set(top_600)

# removing all the books that are not in the top 600

new_ratings = ratings[ratings["ISBN"].isin(top_600)]

In [19]:
# make a dataframe in which the index is ISBN and the columns are the userIDs and the values are the ratings given by the user

new_ratings = new_ratings.pivot(index="User-ID", columns="ISBN", values="bookRating")

In [20]:
new_ratings.fillna(0, inplace=True)

In [21]:
new_ratings.head()

ISBN,000649840X,002542730X,0028604199,006000438X,0060008032,0060096195,006016848X,0060173289,0060175400,0060175966,...,1592400876,1844262553,1857022424,1878424319,1931561648,3257229534,3404148665,3423202327,3442541751,3492045170
User-ID,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# standardizing the ratings so that the mean of each row is 0 and the values are between -1 and 1

def standardize(row):
    new_row = (row - row.mean())/(row.max() - row.min())
    return new_row

new_ratings = new_ratings.apply(standardize)

In [23]:

new_ratings

ISBN,000649840X,002542730X,0028604199,006000438X,0060008032,0060096195,006016848X,0060173289,0060175400,0060175966,...,1592400876,1844262553,1857022424,1878424319,1931561648,3257229534,3404148665,3423202327,3442541751,3492045170
User-ID,,,,,,,,,,,,,,,,,,,,,
9,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
16,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
26,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
39,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
42,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278836,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
278843,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,0.898239,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059
278844,-0.001247,-0.002357,-0.001024,-0.001004,-0.000996,-0.00169,-0.001553,-0.001761,-0.002565,-0.001086,...,-0.001867,-0.001686,-0.001075,-0.001824,-0.001235,-0.000988,-0.001098,-0.001063,-0.000953,-0.001059


In [26]:
from sklearn.metrics.pairwise import cosine_similarity
# making a matrix in which the index and columns are the ISBNs and the values are the similarity between the two books

item_similarity = cosine_similarity(new_ratings.T)  # .T is used to transpose the matrix because we want the similarity between the ISBNs and not the users.

In [28]:

item_similarity_df = pd.DataFrame(item_similarity, index=new_ratings.columns, columns=new_ratings.columns)
item_similarity_df.head()

ISBN,000649840X,002542730X,0028604199,006000438X,0060008032,0060096195,006016848X,0060173289,0060175400,0060175966,...,1592400876,1844262553,1857022424,1878424319,1931561648,3257229534,3404148665,3423202327,3442541751,3492045170
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,1.000000,0.057869,-0.001379,0.066046,0.027202,-0.001766,0.042188,-0.001828,0.018715,-0.001409,...,0.020248,0.018746,0.055158,0.013703,0.028201,-0.001350,0.030416,0.025536,0.024753,0.029912
002542730X,0.057869,1.000000,0.244047,0.009532,0.013646,-0.002424,0.011793,-0.002509,0.022060,-0.001935,...,0.023771,0.008793,0.012553,-0.002525,0.014157,-0.001853,0.015420,0.012732,0.012312,0.015080
0028604199,-0.001379,0.244047,1.000000,-0.001261,-0.001277,0.021499,-0.001619,-0.001648,0.016111,-0.001271,...,0.022599,-0.001546,-0.001282,-0.001658,-0.001313,-0.001217,-0.001252,-0.001284,-0.001277,-0.001323
006000438X,0.066046,0.009532,-0.001261,1.000000,0.023071,-0.001614,0.037972,-0.001670,0.015801,0.056602,...,0.017125,0.015853,0.021370,0.013442,0.023919,-0.001234,0.025810,0.021652,0.020986,0.025375
0060008032,0.027202,0.013646,-0.001277,0.023071,1.000000,0.034914,0.062377,-0.001692,0.022187,-0.001305,...,0.046627,0.022127,0.048832,-0.001703,0.033027,-0.001250,0.035578,0.029930,0.029020,0.035011


In [29]:

def get_similar_books(book_code, user_rating):
    similar_score = item_similarity_df[book_code]*(user_rating-5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

l = get_similar_books("059035342X", 10).head(10)

#showing image of the book cover
display(Image(url=books[books["ISBN"] == "059035342X"]["imageURLM"].values[0]))
for i in l.index:
    display(Image(url=books[books["ISBN"] == i]["imageURLM"].values[0]))
    print(books[books["ISBN"] == i]["bookTitle"].values[0])

for multiple inputs by a user

In [34]:
def get_recommendations(book_ratings):
    recommendations = pd.Series()  # Empty Series to store the recommendations
    
    for book, rating in book_ratings.items():
        similar_scores = item_similarity_df[book] * (rating - 5)
        similar_scores = similar_scores.sort_values(ascending=False)
        recommendations = pd.concat([recommendations, similar_scores])
        
    recommendations = recommendations.groupby(recommendations.index).sum()
    recommendations = recommendations.sort_values(ascending=False)
    
    l = []
    for i in recommendations.index:
        if i not in book_ratings.keys():
            l.append(i)
        if len(l)==10:
            break

    return l

In [36]:
d = {
    "059035342X": 9,
    "0345370775": 10,
    "044021145X": 8,
    "0440214041": 10,
    "0440211727": 7,
}

for i in d:
    display(Image(url=books[books["ISBN"] == i]["imageURLM"].values[0]))
    print(books[books["ISBN"] == i]["bookTitle"].values[0])

print("-------------------------------------------")
l = get_recommendations(d)

for i in l:
    display(Image(url=books[books["ISBN"] == i]["imageURLM"].values[0]))
    print(books[books["ISBN"] == i]["bookTitle"].values[0])

Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))


Jurassic Park


The Firm


The Pelican Brief


A Time to Kill
-------------------------------------------


...

Postmortem


Harry Potter and the Prisoner of Azkaban (Book 3)


The Runaway Jury


Harry Potter and the Goblet of Fire (Book 4)


The Lost World


In [39]:
# making a text file in which write the names of 600 books that we have used

f = open("books.txt", "w")
for i in top_600:
    name = books[books["ISBN"] == i]["bookTitle"].values[0]
    f.write(name+" "+i)
    f.write("\n")
f.close()

In [41]:
with open('books.txt', 'r') as f:
    data = f.read()

In [43]:
data

'E Is for Evidence: A Kinsey Millhone Mystery (Kinsey Millhone Mysteries (Paperback)) 0553279556\nAngels 0060008032\nWhile I Was Gone 0345443284\nScarlet Feather 0451203771\nBehind the Scenes at the Museum 0552996181\nG Is for Gumshoe (Kinsey Millhone Mysteries (Paperback)) 0449219364\nThe Reader 0375707972\nDaddy\'s Little Girl 0743206045\nIrish Hearts 0373484003\nThe Princess Diaries 0380814021\nA Case of Need 0451183665\nThe Lovely Bones 0316168815\nMemoirs of a Geisha Uk 0099771519\nBlackberry Wine 0552998001\nA Prayer for Owen Meany 0345361792\nInto the Wild 0385486804\nCharlotte\'s Web (Trophy Newbery) 0064400557\nHearts In Atlantis 0671024248\nFull House (Janet Evanovich\'s Full Series) 0312983271\nBody of Evidence (Kay Scarpetta Mysteries (Paperback)) 0380717018\nThe Poisonwood Bible: A Novel 0060930535\nHere on Earth 0425167313\nWhat Looks Like Crazy On An Ordinary Day 038079487X\nThe Perks of Being a Wallflower 0671027344\nVoyager 0440217563\nThe Rule of Four 0385337116\nTo t

demo on a real user

In [46]:

# ratings by (my dear friend)

d = {
    "0439139597": 7,
    "0345391802": 8,
    "0590353403": 6,
    "0439064864": 3,
    "0316769487": 8,
    "0439136350": 5,
    "059035342X": 8,
    "0684801523": 10,
    "0439136369": 5,
    "043935806X": 4,
    "0439064872": 3,
    "0804111359": 9,
    "0451526341": 9,
    "0156628708": 9,
    "0451524934": 7
}

for i in d:
    display(Image(url=books[books["ISBN"] == i]["imageURLM"].values[0]))
    print(books[books["ISBN"] == i]["bookTitle"].values[0])

Harry Potter and the Goblet of Fire (Book 4)


The Hitchhiker's Guide to the Galaxy


Harry Potter and the Sorcerer's Stone (Book 1)


Harry Potter and the Chamber of Secrets (Book 2)


The Catcher in the Rye


...

Harry Potter and the Chamber of Secrets (Book 2)


Secret History


Animal Farm


Mrs Dalloway


1984


In [47]:
Image(url=books[books["ISBN"] == i]["imageURLM"].values[0])

In [48]:
l = get_recommendations(d)

for i in l:
    display(Image(url=books[books["ISBN"] == i]["imageURLM"].values[0]))
    print(books[books["ISBN"] == i]["bookTitle"].values[0])

# Samyak was satisfied with the recommendations

C:\Users\siddh\AppData\Local\Temp\ipykernel_15368\624209120.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  recommendations = pd.concat([recommendations, similar_scores])


Lord of the Flies


To Kill a Mockingbird


The Fellowship of the Ring (The Lord of the Rings, Part 1)


The Grapes of Wrath: John Steinbeck Centennial Edition (1902-2002)


...

The World According to Garp


The Stone Diaries


Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players


Anne of Green Gables (Anne of Green Gables Novels (Paperback))


Tis: A Memoir


In [49]:
# make a pickle file in which we store the item_similarity_df

import pickle

pickle.dump(item_similarity_df, open("item_similarity_df.pkl", "wb"))

Content Based Recommendations
Created a numpy array called main_matrix with dimensions (50000, 900) to represent ratings for each book across 900 unique genres.

Populated main_matrix by assigning average ratings to the corresponding positions based on the book's genres.

Defined the get_recommendations(s) function to generate book recommendations based on the user's genre preferences.

Initialized a numpy array called like (shape: (900,)) with zeros to represent the user's genre preferences.

Calculated recommendation scores by performing matrix multiplication between main_matrix and like.

Sorted the recommendation scores and obtained the top 5 book recommendations based on the highest scores.

Printed the titles of the top 5 recommended books from the original dataset.

In [51]:
data = pd.read_csv("books_1.Best_Books_Ever.csv")
data
l=[]
df = data[["isbn","genres","rating"]]
df["genres"] = df["genres"].apply(eval)
df["genres"][0]
s = set()
for i in df["genres"]:
    for j in i:
        s.add(j)
with open("genres_order.txt", "w", encoding="utf-8") as f:
    d = defaultdict(int)
    j = 0

    for i in s:
        d[i] = j
        f.write(i + "\n")
        j += 1
    f.close()
df
main_matrix = np.zeros((52478, 982))
for i in range(len(df)):
    for j in df["genres"][i]:
        main_matrix[i][d[j]] = df["rating"][i]
def get_recommendations(s):
    like = np.zeros((982,))
    for i in s:
        like[d[i]] = 1
    
    ans = np.dot(main_matrix, like)
    
    for i in range(len(ans)):
        l.append((ans[i], i))
    l.sort(reverse=True)

    for i in l[:5]:
        print(data.loc[i[1], "title"])
    return l    

get_recommendations(["Fiction", "Romance", "Magic","Vampires","Action"])
with open('genre_matrix.pkl', 'wb') as file:
    pickle.dump(main_matrix, file)
with open("booktitle_chatbot.txt","w",encoding="utf-8") as file:
    for i in data["title"]:
        file.write(i + "\n")
        
    

C:\Users\siddh\AppData\Local\Temp\ipykernel_15368\1491645833.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["genres"] = df["genres"].apply(eval)


Act of Passion
A Ride of Peril
A Court of Thorns and Roses eSampler
A Meet of Tribes
The Morganville Vampires, #1-9


In [54]:
for i in l[:5]:
        print(data.loc[i[1], "coverImg"])

https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1511148105l/36627208._SY475_.jpg
https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1496170495l/35268661._SY475_.jpg
https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1421578982l/23757205.jpg
https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1493946355l/35065263._SY475_.jpg
https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1344709473l/10786926.jpg


In [ ]:
list_genere = [x for x in input().split(",")]
rec_books=pd.read_csv("books_1.Best_Books_Ever.csv")

list_recomm = []

get_recommendations(list_genere)
with open('genre_matrix.pkl', 'wb') as file:
    pickle.dump(main_matrix, file)
with open("booktitle_chatbot.txt","w",encoding="utf-8") as file:
    for i in data["title"]:
        file.write(i + "\n")
        list_recomm.append(i)

In [ ]:
list_recomm[0]

In [ ]:
rec_books=pd.read_csv("books_1.Best_Books_Ever.csv")
list_url=[]
for i in list_recomm:
    image_url = rec_books[rec_books["title"] == i]["coverImg"].values[0]
    list_url.append(image_url)
    

In [ ]:
list_url

In [ ]:
rec_books=pd.read_csv("books_1.Best_Books_Ever.csv")
image_url = rec_books[rec_books["title"] == "Harry Potter and the Goblet of Fire"]["coverImg"].values[0]
Image(image_url)

In [ ]:
image_url